In [1]:
# Step 1: Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Step 2: Load and preprocess dataset (define your path here)
data_dir = "Train"
img_size = 32
num_classes = 43  # GTSRB has 43 classes

X = []
y = []

In [3]:
for label in range(num_classes):
    path = os.path.join(data_dir, str(label))
    for img_file in os.listdir(path):
        try:
            img = cv2.imread(os.path.join(path, img_file))
            img = cv2.resize(img, (img_size, img_size))
            X.append(img)
            y.append(label)
        except Exception as e:
            continue

X = np.array(X) / 255.0
y = to_categorical(y, num_classes)

print(f"✅ Loaded {len(X)} images")

✅ Loaded 39209 images


In [4]:
# Step 3: Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Step 4: Try loading pre-trained model
model_path = "traffic_sign_model.h5"

In [6]:
if os.path.exists(model_path):
    model = load_model(model_path)
    print("✅ Loaded pre-trained model!")
else:
    print("❌ No pre-trained model found. Training a new model...")
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', patience=3)
    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[es])
    model.save(model_path)
    print("✅ Model trained and saved.")

✅ Loaded pre-trained model!


In [7]:
# Step 5: Evaluate
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"📊 Validation Accuracy: {val_acc:.4f}")

246/246 ━━━━━━━━━━━━━━━━━━━━ 23s 41ms/step - accuracy: 0.9908 - loss: 0.0359
📊 Validation Accuracy: 0.9908


In [8]:
# Step 6: Show confusion matrix
preds = model.predict(X_val)
y_pred = np.argmax(preds, axis=1)
y_true = np.argmax(y_val, axis=1)

cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(cm)

246/246 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step

Confusion Matrix:
[[ 32   4   0 ...   0   0   0]
 [  0 491   3 ...   0   0   0]
 [  0   0 445 ...   0   0   0]
 ...
 [  0   0   0 ...  60   0   0]
 [  0   0   0 ...   0  46   0]
 [  0   0   0 ...   0   0  41]]


In [9]:
# Step 7: Visualize Accuracy & Loss Curves
if 'history' in locals():
    # Accuracy Plot
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()

    # Loss Plot
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()
